In [161]:
import numpy as np
import pandas as pd
import folium
import urllib
import json
import socket
from ipwhois import IPWhois
import pycountry


In [180]:
# Load data
# 20150218230000.export.CSV  20150218230000.gkg.csv  20150218230000.mentions.CSV
DATA_PATH = "data/"
MAP_PATH = DATA_PATH + "world.geo.json/countries/"
COUNTRY_CODE_DATA = DATA_PATH + "country-codes/data/country-codes.csv"

export = pd.read_csv(DATA_PATH + "20150218230000.export.CSV",sep='\t', header=None)
gkg = pd.read_csv(DATA_PATH + "20150218230000.gkg.csv",sep='\t', header=None)
mention = pd.read_csv(DATA_PATH + "20150218230000.mentions.CSV", sep="\t", header=None)

In [191]:
def isNaN(num):
    return num != num

def get_export_names():
    file = open(DATA_PATH + "event_table_name", "r")
    names = file.readlines()[0].split(" ")
    return names

def get_mentions_names():
    file = open(DATA_PATH + "mentions_table_name", "r")
    names = file.readlines()[0].split(" ")
    return names

def get_map_site():
    file = pd.read_csv(COUNTRY_CODE_DATA)
    return dict(zip(file['TLD'], file['ISO3166-1-Alpha-3'])), dict(zip(file['ISO3166-1-Alpha-3'], file['TLD']))


In [193]:
a,s = get_map_site()
a[".ch"]

'CHE'

In [164]:
print(get_export_names())
print(len(get_export_names()))
print(get_mentions_names())
print(len(get_mentions_names()))

['GlobalEventID', 'Day', 'MounthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religioni1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religioni1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_Fullname', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_Fullname', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_ADM2Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_Fullname', 'ActionGeo_CountryCod

In [165]:
export_d = pd.read_csv(DATA_PATH + "20150218230000.export.CSV",sep='\t', names=get_export_names())
print(len(export_d))
export_d.head()


2271


,GlobalEventID,Day,MounthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_Fullname,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,410412347,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,4,"Waterkloof, Free State, South Africa",SF,SF03,77359,-30.3098,25.2971,-1299321,20150218230000,http://www.dailymaverick.co.za/article/2015-02...
1,410412348,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,4,"Bengaluru, Karnataka, India",IN,IN19,70159,12.9833,77.5833,-2090174,20150218230000,http://timesofindia.indiatimes.com/city/bengal...
2,410412349,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,4,"Great Southern, Victoria, Australia",AS,AS07,5387,-36.0667,146.4830,-1576477,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...
3,410412350,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,1,New Zealand,NZ,NZ,NaN,-41.0000,174.0000,NZ,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...
4,410412351,20140218,201402,2014,2014.1315,NaN,NaN,NaN,NaN,NaN,...,2,"Idaho, United States",US,USID,NaN,44.2394,-114.5100,ID,20150218230000,http://www.eastidahonews.com/2015/02/neil-patr...


In [166]:
gkg.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,20150218230000-0,20150218230000,2,BBC Monitoring,as listed in Russian /BBC Monitoring/(c) BBC,"ARREST#400#political#4#Rossiya, Orenburgskaya ...","ARREST#400#political#4#Rossiya, Orenburgskaya ...",TERROR;REBELS;TAX_ETHNICITY;TAX_ETHNICITY_UKRA...,"TAX_FNCACT,2011;TAX_FNCACT,3449;TAX_FNCACT,381...","4#Budapest, Budapest, Hungary#RS#HU05#47.5#19....",...,"wc:693,c1.1:1,c1.2:1,c12.1:43,c12.10:58,c12.11...",NaN,NaN,NaN,NaN,1332|134||prisoners are treated humanely ; the...,"Channel One,628;Channel One,755;Channel One,14...","3,channels,476;3,channels reported,824;1,corre...",NaN,NaN
1,20150218230000-1,20150218230000,2,BBC Monitoring,"Al-Sharq al-Awsat website, London/BBC Monitori...",NaN,NaN,TAX_FNCACT;TAX_FNCACT_ENVOY;TAX_ETHNICITY;TAX_...,"IDEOLOGY,6790;IDEOLOGY,10592;KILL,1960;KILL,13...",1#Qatar#SA#QA#25.5#51.25#QA;1#Syria#SA#SY#35#3...,...,"wc:2376,c1.2:1,c1.4:7,c12.1:163,c12.10:276,c12...",NaN,NaN,NaN,NaN,332|26||greatly complicate matters#4494|37||ma...,"Emrullah Isler,83;Arab Spring,3571;Development...","2,fronts at the political,1455;2,main strategi...",NaN,NaN
2,20150218230000-2,20150218230000,1,wjon.com,http://wjon.com/wjon-news-on-demand-wednesday-...,NaN,NaN,MANMADE_DISASTER;MANMADE_DISASTER_WITHOUT_POWE...,"MANMADE_DISASTER,91;POWER_OUTAGE,91;",NaN,...,"wc:93,c12.1:7,c12.10:10,c12.12:5,c12.13:4,c12....",http://wac.450F.edgecastcdn.net/80450F/wjon.co...,http:/wac.450F.edgecastcdn.net/80450F/wjon.com...,NaN,https://youtube.com/channel/;https://youtube.c...,NaN,"Waite Park,125;City Food,233;Adobe Flash Playe...","500,people were without power,51;",NaN,NaN
3,20150218230000-3,20150218230000,1,wjol.com,http://www.wjol.com/common/more.php?m=15&r=3&i...,NaN,NaN,LEADER;TAX_FNCACT;TAX_FNCACT_GOVERNOR;TAX_POLI...,"TAX_FNCACT,131;TAX_POLITICAL_PARTY,664;TAX_FNC...","2#Wisconsin, United States#US#USWI#44.2563#-89...",...,"wc:103,c12.1:4,c12.10:13,c12.12:7,c12.13:6,c12...",NaN,NaN,NaN,NaN,NaN,"Governor Rauner,41;Speaker Michael Madigan,158...",NaN,NaN,NaN
4,20150218230000-4,20150218230000,1,straitstimes.com,http://www.straitstimes.com:80/news/world/unit...,NaN,NaN,DRONES;TAX_WORLDMAMMALS;TAX_WORLDMAMMALS_MICE;...,"TAX_FNCACT,634;TAX_FNCACT,1248;TAX_FNCACT,907;...","1#United States#US#US#38#-97#US;3#Miami, Flori...",...,"wc:255,c12.1:10,c12.10:28,c12.12:11,c12.13:7,c...",http://www.straitstimes.com/sites/straitstimes...,NaN,NaN,NaN,NaN,"United States,583;Columbia University Medical ...","5,weeks,1194;",NaN,NaN


In [167]:
mention_d = pd.read_csv(DATA_PATH + "20150218230000.mentions.CSV", sep="\t", names=get_mentions_names())
mention_d[200:300]

,GlobalEventId,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,ActorCharOffset,Actor2CharOffset,ActionCharOffset,InRawTest,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationinfo,Extras
200,410333265,20150218170000,20150218230000,1,thesudburystar.com,http://www.thesudburystar.com/2015/02/18/quebe...,6,1011,1060,1033,1,80,2055,-1.812689,NaN,NaN
201,410412408,20150218230000,20150218230000,1,dailymail.co.uk,http://www.dailymail.co.uk/indiahome/indianews...,6,1703,-1,1743,0,20,2098,-4.273504,NaN,NaN
202,403732612,20150119044500,20150218230000,1,dailymail.co.uk,http://www.dailymail.co.uk/indiahome/indianews...,6,1703,-1,1743,1,80,2098,-4.273504,NaN,NaN
203,403890177,20150119203000,20150218230000,1,carsonnow.org,http://carsonnow.org/xfeed/02/18/2015/nevada-l...,5,1662,-1,1694,0,80,4037,-5.422648,NaN,NaN
204,403890177,20150119203000,20150218230000,1,carsonnow.org,http://carsonnow.org/xfeed/02/18/2015/drought-...,6,1635,-1,1667,0,80,4037,-5.760000,NaN,NaN
205,410412409,20150218230000,20150218230000,1,oregonlive.com,http://www.oregonlive.com/politics/index.ssf/2...,4,437,-1,488,0,20,3876,-3.164557,NaN,NaN
206,403829985,20150119133000,20150218230000,1,dailymail.co.uk,http://www.dailymail.co.uk/wires/ap/article-29...,2,341,-1,309,0,20,838,-10.218978,NaN,NaN
207,403829985,20150119133000,20150218230000,1,startribune.com,http://www.startribune.com/politics/292469151....,3,361,-1,329,0,20,827,-10.791367,NaN,NaN
208,403829985,20150119133000,20150218230000,1,wral.com,http://www.wral.com/former-cia-officer-suing-a...,2,372,-1,340,0,20,858,-9.285714,NaN,NaN
209,403829985,20150119133000,20150218230000,1,abc-7.com,http://www.abc-7.com/story/28142179/former-cia...,2,367,-1,335,0,20,890,-9.027778,NaN,NaN


In [168]:
# Funny visulalisation

m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=2,
    tiles='Stamen Terrain'
)
    
lat = export_d['ActionGeo_Lat']
long = export_d['ActionGeo_Long']
gold = export_d['GoldsteinScale']
src = export_d['SOURCEURL\n']


for x,y,z,w in zip(lat[:500],long[:500], gold[:500], src[:500]): # Display 500 elems for shit and giggles
    if isNaN(x) or isNaN(y):
        pass
    else:
        if z > 0:
            if z > 5:
                col = "green"
            else :
                col = "lightgreen" 
        else:
            if z < -5:
                col = "red"
            else:
                col = "pink"
        #print(col)
        folium.Marker(
            location=[x, y],
            popup=w,
            icon=folium.Icon(color=col)
        ).add_to(m)
        
m

In [169]:
#df.loc[df['column_name'] == some_value]


In [170]:
events_id = export_d.loc[export_d['NumMentions'] == max(export_d['NumMentions'])]['GlobalEventID']

references = mention_d.loc[mention_d['GlobalEventId'] == events_id.values[0]]

references

,GlobalEventId,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,ActorCharOffset,Actor2CharOffset,ActionCharOffset,InRawTest,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationinfo,Extras
2147,410412879,20150218230000,20150218230000,1,yahoo.com,http://news.yahoo.com/islamic-state-militants-...,26,6847,6772,6854,1,80,7367,-4.187605,NaN,NaN
2148,410412879,20150218230000,20150218230000,1,kswo.com,http://www.kswo.com/story/28142565/islamic-sta...,27,7145,7070,7152,1,80,7596,-3.793382,NaN,NaN
2149,410412879,20150218230000,20150218230000,1,sfgate.com,http://www.sfgate.com/news/world/article/Islam...,32,10557,10482,10564,1,80,11165,-2.133633,NaN,NaN
2150,410412879,20150218230000,20150218230000,1,kxxv.com,http://www.kxxv.com/story/28142565/islamic-sta...,27,7141,7066,7148,1,80,7596,-3.787268,NaN,NaN
2151,410412879,20150218230000,20150218230000,1,ap.org,http://hosted2.ap.org/CAANR/0260ea4c3e85456b80...,31,10087,10012,10094,1,80,10716,-2.153667,NaN,NaN
2152,410412879,20150218230000,20150218230000,1,beaumontenterprise.com,http://www.beaumontenterprise.com/news/world/a...,26,7104,7029,7111,1,80,7603,-3.799515,NaN,NaN
2153,410412879,20150218230000,20150218230000,1,ap.org,http://hosted2.ap.org/APDEFAULT/cae69a7523db45...,31,10157,10082,10164,1,80,10824,-2.136259,NaN,NaN
2154,410412879,20150218230000,20150218230000,1,yahoo.com,https://uk.news.yahoo.com/islamic-state-milita...,26,7104,7029,7111,1,80,7601,-3.799515,NaN,NaN
2155,410412879,20150218230000,20150218230000,1,washingtonpost.com,http://www.washingtonpost.com/world/middle_eas...,28,7609,7534,7616,1,80,8183,-3.484848,NaN,NaN
2156,410412879,20150218230000,20150218230000,1,therepublic.com,http://www.therepublic.com/view/story/5563550e...,27,7595,7520,7602,1,80,8068,-3.435115,NaN,NaN


In [204]:
print (socket.gethostbyname('postzambia.com')) 

192.64.119.171


In [205]:
from ipwhois import IPWhois
import pycountry

# TODO put try and catch to make the function robust to unknown values
def src_to_country(web, f_map, color, ip = True): # Ture -> Country by IP, False -> Country by www.[..].country
    if ip:
        obj = IPWhois(socket.gethostbyname(web))
        results = obj.lookup_rdap(obj)
        country = pycountry.countries.get(alpha_2=results['asn_country_code'])
        layout = MAP_PATH + country.alpha_3 + ".geo.json"
        print(layout)

        folium.GeoJson(layout).add_to(f_map)
        
    else:
        site, _ = get_map_site() # get dictionnary
        country = site[str(".") + web.split('.')[-1]] # get web extension
        layout = MAP_PATH + country + ".geo.json"
        print(layout)

        folium.GeoJson(layout).add_to(f_map)
        
def tone_to_color(tone): # Tone is between -100 , 100 but -10 , 10 for 99% du time 
    pass

In [208]:
src_to_country('dailymail.co.uk', m, "ROT", False) # ENGLAND
src_to_country('dailytimes.com.pk', m, "Sherlock", False) # No color encoding rn # PAKISTAN 
src_to_country('kswo.com', m, "Watson") # cast by ip # USa

data/world.geo.json/countries/GBR.geo.json
data/world.geo.json/countries/PAK.geo.json
data/world.geo.json/countries/USA.geo.json


In [209]:
m # Show for example two source on the map